In [286]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from scipy.stats import norm
from sklearn.utils import check_X_y
from sklearn.base import clone

class BoostedOrdinal(BaseEstimator, RegressorMixin):
    def __init__(self, base_learner = DecisionTreeRegressor(), max_iter=100, lr_g = 1.0, lr_theta = 1e-2):
        self.base_learner = base_learner
        self.max_iter = max_iter
        self.lr_g = lr_g
        self.lr_theta = lr_theta

    def predict(self, X):
        check_array(X)
        return np.array([learner.predict(X) + self.path['intercept'][p] for p, learner in enumerate(self.path['learner'])])
    
    def _probabilities(g, theta, y = None):
        probs = np.array([np.diff(norm.cdf(BoostedOrdinal._pad_thresholds(theta - x))) for x in g])

        if y is None:
            return probs
        
        loglike = sum([np.log(probs[n, yn]) for n, yn in enumerate(y)])
        return probs, loglike
    
    def _check_loss_change(loss):
        x = np.diff(loss)
        return (x[::2], x[1::2]) # (g, theta)
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        ylist = BoostedOrdinal._validate_ordinal(y)

        g_init, theta_init = BoostedOrdinal._initialize(y)
        loss_init = BoostedOrdinal._loss_function(X, y, g_init, theta_init)

        g, theta = g_init, theta_init
        loss_all = []
        learner_all = []
        intercept_all = []
        g_all = []
        theta_all = []

        loss_all.append(loss_init)
        g_all.append(g_init)
        theta_all.append(theta_init)

        for p in range(self.max_iter):
            
            # update regression function
            dg = BoostedOrdinal._derivative_g(X, y, theta, g)
            weak_learner, h, intercept = BoostedOrdinal._fit_weak_learner(X, -dg, clone(self.base_learner))
            g = BoostedOrdinal._update_g(g, h, lr = self.lr_g)
            
            # update loss
            loss_all.append(
                BoostedOrdinal._loss_function(X, y, g, theta)
            )
            
            # update threshold vector
            dtheta = BoostedOrdinal._derivative_threshold(X, ylist, theta, g)
            theta = BoostedOrdinal._update_thresh(theta, dtheta, lr = self.lr_theta)

            # update loss
            loss_all.append(
                BoostedOrdinal._loss_function(X, y, g, theta)
            )
            
            learner_all.append(weak_learner)
            intercept_all.append(intercept)
            g_all.append(g)
            theta_all.append(theta)

        self.n_iter = self.max_iter # in future, we may exit before reaching max_iter
        self.final = {'g': g, 'theta': theta, 'loss': loss_all[-1]}
        self.path = {'g': np.array(g_all), 'theta': np.array(theta_all), 'loss': np.array(loss_all), 'learner': learner_all, 'intercept': np.array(intercept_all)}
        
        return self
    
    def _validate_ordinal(arr):
        """
        Check if the unique values in a numpy integer vector are 0, 1, ..., M with M >= 2.
    
        Parameters:
        arr (numpy.ndarray): Input numpy integer vector.
    
        Returns:
        bool: True if unique values are 0, 1, ..., M with M >= 2, False otherwise.
        """
        if not isinstance(arr, np.ndarray):
            raise ValueError("Input must be a numpy array")
        if arr.dtype.kind not in {'i', 'u'}:
            raise ValueError("Input array must contain integers")
    
        unique_values = np.unique(arr)
        
        if unique_values[0] != 0:
            return []
        
        M = unique_values[-1]

        if M < 2:
            return []
        
        expected_values = np.arange(M + 1)

        if np.array_equal(unique_values, expected_values):
            #return M + 1
            return [np.where(arr == m) for m in unique_values]
        else:
            return []
    
    def _initialize(y):
        return (BoostedOrdinal._initialize_g(y), BoostedOrdinal._initialize_thresholds(y))
    
    def _initialize_g(y):
        return np.zeros(y.size)
    
    def _initialize_thresholds(y):
        # Calculate the initial threshold vector
        n_samples = len(y)
        n_class = np.max(y) + 1
        P = np.array([np.sum(y == i) for i in range(n_class)]) / n_samples
        return norm.ppf(np.cumsum(P[:-1]))
    
    def _pad_thresholds(theta):
        return np.insert(theta, [0, theta.size], [-np.inf, np.inf])
    
    def _derivative_threshold(X, ylist, thresh, g):
        thresh_padded = BoostedOrdinal._pad_thresholds(thresh)
        M = len(thresh)
        ret = []
        for m in range(M):
            S_m = ylist[m]
            S_mp1 = ylist[m+1]
            v1 = np.sum(norm.pdf(thresh_padded[m+1] - g[S_m]) / (norm.cdf(thresh_padded[m+1] - g[S_m]) - norm.cdf(thresh_padded[m] - g[S_m])))
            v2 = np.sum(norm.pdf(thresh_padded[m+1] - g[S_mp1]) / (norm.cdf(thresh_padded[m+2] - g[S_mp1]) - norm.cdf(thresh_padded[m+1] - g[S_mp1])))
            ret.append(-v1 + v2)
        return np.array(ret)

    def _derivative_g(X, y, thresh, g):
        thresh_padded = BoostedOrdinal._pad_thresholds(thresh)
        ret = (norm.pdf(thresh_padded[y+1] - g) - norm.pdf(thresh_padded[y] - g)) / (norm.cdf(thresh_padded[y+1] - g) - norm.cdf(thresh_padded[y] - g))
        return ret

    def _fit_weak_learner(X, pseudo_resids, learner):
        learner.fit(X, pseudo_resids)
        pred = learner.predict(X)
        intercept = -np.mean(pred) # we could also perform intercept adjustment in _update_g but mathematically the effect is the same
        return (learner, pred + intercept, intercept)
    
    # replace with more sophisticated version that performs line search
    def _update_g(g, h, lr = 1e-1):
        return g + lr * h
    
    # we need to check if updated thresh is valid (must be sorted) and handle invalid ones
    def _update_thresh(thresh, dthresh, lr = 1e-3):
        new_thresh = thresh - lr * dthresh
        if not np.all(np.diff(new_thresh)):
            raise ValueError("updated threshold vector invalid (must have strict ascending order)")
        return new_thresh
    
    # this can be fused with _probabilities, though this is likely more efficient is the goal is only loss and not the prob matrix
    def _loss_function(X, y, g, theta):
        theta_padded = BoostedOrdinal._pad_thresholds(theta)
        return -np.sum(np.log(norm.cdf(theta_padded[y + 1] - g) - norm.cdf(theta_padded[y] - g)))

In [196]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create a sample dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, n_informative=5, random_state=1)
indices = BoostedOrdinal._validate_ordinal(y)
#print(len(indices))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [287]:
model = BoostedOrdinal(max_iter = 10, lr_g = 1e-1, lr_theta = 1e-3, base_learner = DecisionTreeRegressor(max_depth = 3)).fit(X_train, y_train)

In [282]:
model.path['loss']

array([878.85487184, 853.07589248, 852.9936132 , 830.61493802,
       830.53102021, 810.96326189, 810.88391969, 794.58732095,
       794.52296859, 778.94443116, 778.87888521, 762.28788039,
       762.21386072, 749.49485681, 749.43077129, 735.75591784,
       735.68958443, 723.53156407, 723.47251361, 712.69965495,
       712.64283255])

In [285]:
_ , loglike = BoostedOrdinal._probabilities(model.final['g'], model.final['theta'], y_train)
#_ , loglike = BoostedOrdinal._probabilities(model.path['g'][0], model.path['theta'][0], y_train)
loglike

-712.6428325463814

In [277]:
dloss_g, dloss_theta = BoostedOrdinal._check_loss_change(model.path['loss'])
np.all(dloss_g < 0), np.all(dloss_theta < 0)

(True, True)

In [188]:
my_indices = BoostedOrdinal._validate_ordinal(y_train)
theta_init = BoostedOrdinal._initialize_thresholds(y_train)# + 1
g_init = np.zeros(X_train.shape[0])

my_loss_init = BoostedOrdinal._loss_function(X_train, y_train, g_init, theta_init)

#BoostedOrdinal._derivative_threshold(X_train, my_indices, theta_init, g_init)
#my_dthresh = BoostedOrdinal._derivative_threshold(X_train, my_indices, theta_init, g_init)
#print(theta_init)
#BoostedOrdinal._update_thresh(theta_init, my_dthresh)
my_pseudo_resids = -BoostedOrdinal._derivative_g(X_train, y_train, theta_init, g_init)
#my_pseudo_resids#[3:5]
my_loss_init

878.8548718382214

In [189]:
from sklearn.tree import DecisionTreeRegressor

my_weak_learner, my_h = BoostedOrdinal._fit_weak_learner(X_train, my_pseudo_resids, DecisionTreeRegressor(max_depth = 3))
my_g = BoostedOrdinal._update_g(g_init, my_h, lr = 1e0)
BoostedOrdinal._loss_function(X_train, y_train, g_init + my_g, theta_init)

706.1121645706666

In [190]:
my_dthresh = BoostedOrdinal._derivative_threshold(X_train, my_indices, theta_init, my_g)
my_theta = BoostedOrdinal._update_thresh(theta_init, my_dthresh)
BoostedOrdinal._loss_function(X_train, y_train, my_g, my_theta)

698.6227248850801

In [194]:
BoostedOrdinal._initialize_g(y[:5])

array([0., 0., 0., 0., 0.])

In [255]:
u = 1
if u:
    print('yes')

yes


In [272]:
y[1::2]

array([0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 2, 1, 1, 2, 1, 0, 2, 0, 1,
       2, 0, 0, 2, 0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 2, 1, 1, 1, 2, 1,
       2, 0, 2, 0, 1, 2, 2, 1, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1,
       2, 2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 0, 1, 0, 1, 2,
       0, 1, 0, 1, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 0, 0, 1, 0, 1, 2, 2, 0,
       0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 1, 1, 1, 0, 2, 0, 0, 1, 1, 1, 2, 1,
       2, 1, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2,
       1, 2, 1, 1, 1, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 2, 2, 0, 2, 2, 1, 0, 2, 0, 1, 1, 1, 0, 2, 0, 0, 1,
       1, 1, 1, 0, 2, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 2,
       0, 2, 2, 0, 1, 1, 2, 2, 0, 0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 0, 2,
       1, 0, 1, 0, 0, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 1,
       0, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2,